# Pipeline preprocesamiento
Se listan una serie de pasos que se ejecutan sobre las columnas de manera seriada. <br>
Para armar un pipeline con métodos propios, es necesario que sea una clase que herede de las clases base de sklearn. <br>
Esto debería quedar arriba.

In [1]:
import pandas as pd

In [2]:
df_temp = pd.read_csv('BankChurners.csv')

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
class CategoricalMapper(BaseEstimator, TransformerMixin):
    def __init__(self, *, mapper=None):
        self.mapper = mapper
    
    def fit(self, x, y=None):
        return self
        
    def transform(self, x, y=None):                    
        return x.replace(self.mapper)   

In [4]:
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [5]:
df_unknowns = df_temp[(df_temp['Income_Category'] == 'Unknown') & 
                 (df_temp['Marital_Status'] == 'Unknown') & 
                 (df_temp['Education_Level'] == 'Unknown')]

df_unknowns.describe()

,CLIENTNUM,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
count,7.000000e+00,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.00000
mean,7.279316e+08,42.714286,2.285714,34.142857,4.571429,2.285714,2.285714,8257.285714,1354.285714,6903.000000,0.753429,4507.142857,62.571429,0.763000,0.24500
std,2.435630e+07,12.134171,1.976047,7.668737,1.272418,0.951190,0.487950,5378.081248,1011.714010,5770.212879,0.249404,4196.845142,18.301704,0.394679,0.27054
min,7.093752e+08,26.000000,0.000000,18.000000,3.000000,1.000000,2.000000,1801.000000,0.000000,1129.000000,0.499000,1296.000000,31.000000,0.348000,0.00000
25%,7.130736e+08,35.500000,0.500000,35.000000,3.500000,1.500000,2.000000,3920.500000,659.500000,1857.500000,0.586500,2689.000000,55.000000,0.457000,0.05300
50%,7.150076e+08,43.000000,3.000000,36.000000,5.000000,3.000000,2.000000,7270.000000,1528.000000,5435.000000,0.731000,3337.000000,69.000000,0.769000,0.12000
75%,7.376448e+08,49.000000,3.500000,36.000000,5.500000,3.000000,2.500000,12620.500000,2058.000000,11197.000000,0.818500,3877.500000,70.000000,0.915000,0.41000
max,7.697013e+08,61.000000,5.000000,43.000000,6.000000,3.000000,3.000000,15648.000000,2517.000000,15648.000000,1.234000,13784.000000,88.000000,1.480000,0.66900


In [6]:
df_temp.drop(index=df_unknowns.index, inplace=True)

In [7]:
import numpy as np

In [8]:
attrition_mapper = {'Existing Customer': 0, 
                    'Attrited Customer': 1}

gender_mapper = {'F': 0, 
                 'M': 1}

card_mapper = {'Blue': 0,
               'Silver':1,
               'Gold':2,
               'Platinum':3}

income_mapper = {'Less than $40K': 0,
                 '$40K - $60K':1,
                 '$60K - $80K':2,
                 '$80K - $120K':3, 
                 '$120K +':4, 
                 'Unknown': np.nan}

education_mapper = {'Uneducated': 0, 
                    'High School':1, 
                    'College':2, 
                    'Graduate': 3,
                    'Post-Graduate': 4, 
                    'Doctorate': 5, 
                    'Unknown': np.nan}

marital_mapper = {'Single': 0, 
                  'Married':1, 
                  'Divorced':2, 
                  'Unknown': np.nan}

In [9]:
import matplotlib.pyplot as plt
import seaborn as sns

¿Por qué no poner el tratamiento de outliers dentro de un pipeline?: <br>
_Removing samples, does not (yet?) comply with the scikit-learn transformer API. So if you need to do this, you should do it outside any calls to scikit learn, as preprocessing. <br>
As it is now, the transformer API is used to transform the features of a given sample into something new. This can implicitly contain information from other samples, but samples are never deleted. <br>
Another option is to attempt to impute the missing values. But again, if you need to delete samples, treat it as preprocessing before using scikit learn._

In [10]:
def treat_outiers(dataframe, col, **kwargs):
    """
    Treat outliers considering interquartile range.
    """     
    # Get keyword arguments
    column_action = kwargs.pop('column_action', 'remove')

    q1 = dataframe[col].quantile(0.25)
    q3 = dataframe[col].quantile(0.75)
    iqr = q3 - q1
    outlier_threshold = q3 + (iqr *1.5)
    if column_action == 'remove':
        dataframe = dataframe[dataframe[col] < outlier_threshold]
    
    return dataframe

In [11]:
cols = ['Credit_Limit', 'Avg_Open_To_Buy']
for k in range(len(cols)):
    df_temp = treat_outiers(df_temp, cols[k]) 

In [19]:
from sklearn.pipeline import make_pipeline
float_columns = df_temp.select_dtypes(['float64']).columns
num_columns = df_temp.select_dtypes(["int64"]).columns #agrego esto

preprocessor= make_column_transformer(
                #quito el drop CLIENTNUM para que no interfiera con el Scaler num_columns (lo borro después)
                (CategoricalMapper(mapper=attrition_mapper),['Attrition_Flag']),
                (CategoricalMapper(mapper=gender_mapper),['Gender']),     
                (make_pipeline(CategoricalMapper(mapper=marital_mapper),
                               SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
                               ['Marital_Status']),
                (make_pipeline(CategoricalMapper(mapper=education_mapper),
                               SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
                               ['Education_Level']),    
                (make_pipeline(CategoricalMapper(mapper=income_mapper),
                               SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
                               ['Income_Category']),  
                (CategoricalMapper(mapper=card_mapper),['Card_Category']),
                (StandardScaler(),float_columns),
                (StandardScaler(),num_columns), #agrego esto
                remainder='passthrough'
                )

NameError: name 'df_temp' is not defined

In [27]:
df_temp.columns

Index(['CLIENTNUM', 'Attrition_Flag', 'Customer_Age', 'Gender',
       'Dependent_count', 'Education_Level', 'Marital_Status',
       'Income_Category', 'Card_Category', 'Months_on_book',
       'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio'],
      dtype='object')

In [28]:
df_preprocessed = pd.DataFrame(preprocessor.fit_transform(df_temp))
df_preprocessed

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0.0,1.0,1.0,1.0,2.0,0.0,1.842316,1.934200,2.627902,3.822275,...,0.805295,-0.161284,0.525496,0.376958,0.747871,-1.321445,0.499409,-0.474072,-0.976063,-0.974328
1,0.0,0.0,0.0,3.0,0.0,0.0,0.734509,0.807437,3.568206,12.573300,...,2.161689,0.326658,2.059452,0.990279,1.396493,-1.321445,-0.407651,-0.367476,-0.929532,-1.368082
2,0.0,1.0,1.0,3.0,3.0,0.0,-0.473962,-0.182780,8.374710,6.788156,...,-0.682997,0.570629,0.525496,0.008965,0.099250,-1.321445,-2.221772,-1.426087,-0.740873,-1.936836
3,0.0,0.0,1.0,1.0,0.0,0.0,-0.500190,-0.836113,2.947423,6.788156,...,0.835333,-0.771212,1.292474,-0.236363,-0.549372,1.621928,-1.314712,1.657853,-0.967516,-1.936836
4,0.0,1.0,1.0,0.0,2.0,0.0,-0.149738,0.140648,6.462151,7.487735,...,-0.815359,-0.771212,0.525496,-1.830999,0.747871,-1.321445,-2.221772,-1.426087,-1.079888,-1.586833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8615,0.0,1.0,0.0,3.0,1.0,0.0,-0.327837,-0.498233,-0.256914,0.605048,...,0.901978,0.448644,-0.241482,0.499622,-0.549372,-0.340320,0.499409,0.841840,3.560597,2.306948
8616,1.0,1.0,2.0,3.0,1.0,0.0,-0.259395,-0.513433,0.204109,-0.123855,...,-0.773773,-0.649226,-0.241482,-1.340342,0.099250,-0.340320,0.499409,1.252297,1.435977,0.206931
8617,1.0,0.0,1.0,1.0,0.0,0.0,0.023364,0.313325,0.272578,0.441673,...,-0.614478,-0.283270,-1.008461,0.008965,0.747871,0.640804,1.406470,-1.426087,1.919334,-0.186822
8618,1.0,1.0,1.0,3.0,1.0,0.0,-0.008608,0.281431,-1.023764,0.039520,...,-0.590022,-1.991068,-0.241482,0.008965,0.099250,0.640804,0.499409,-1.426087,1.319173,-0.099321


In [29]:
df_preprocessed.columns = ['Attrition_Flag',
                          'Gender',
                          'Marital_Status',
                          'Education_Level',
                          'Income_Category',
                          'Card_Category',
                          'Credit_Limit',
                          'Avg_Open_To_Buy',                          
                          'Total_Amt_Chng_Q4_Q1',
                          'Total_Ct_Chng_Q4_Q1',
                          'Avg_Utilization_Ratio',
                          "CLIENTNUM", #agrego esta por el scaler de columnas numéricas y la borro después
                          'Customer_Age',
                          'Dependent_count',
                          'Months_on_book',
                          'Total_Relationship_Count',
                          'Months_Inactive_12_mon',
                          'Contacts_Count_12_mon',                          
                          'Total_Revolving_Bal',                          
                          'Total_Trans_Amt',
                          'Total_Trans_Ct']

In [30]:
df_preprocessed.drop(columns=["CLIENTNUM",
                              'Dependent_count', 
                              'Education_Level', 
                              'Marital_Status', 
                              'Months_on_book', 
                              'Months_Inactive_12_mon',
                              'Customer_Age'], inplace=True)

In [31]:
df_preprocessed.head()

,Attrition_Flag,Gender,Income_Category,Card_Category,Credit_Limit,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Customer_Age,Total_Relationship_Count,Contacts_Count_12_mon,Total_Revolving_Bal,Total_Trans_Amt,Total_Trans_Ct
0,0.0,1.0,2.0,0.0,1.842316,1.934200,2.627902,3.822275,-0.909964,-0.161284,0.747871,0.499409,-0.474072,-0.976063,-0.974328
1,0.0,0.0,0.0,0.0,0.734509,0.807437,3.568206,12.573300,-0.752494,0.326658,1.396493,-0.407651,-0.367476,-0.929532,-1.368082
2,0.0,1.0,3.0,0.0,-0.473962,-0.182780,8.374710,6.788156,-1.128274,0.570629,0.099250,-2.221772,-1.426087,-0.740873,-1.936836
3,0.0,0.0,0.0,0.0,-0.500190,-0.836113,2.947423,6.788156,1.591659,-0.771212,-0.549372,-1.314712,1.657853,-0.967516,-1.936836
4,0.0,1.0,2.0,0.0,-0.149738,0.140648,6.462151,7.487735,-1.128274,-0.771212,0.747871,-2.221772,-1.426087,-1.079888,-1.586833


*hay columnas numéricas (como Total_Revolving_Bal, Total_Trans_Amt o Total_Trans_Ct) que no están escaladas --> lo agrego en el pipeline de preprocessing